通过使用深度学习框架来简洁的实现线性回归模型生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data

In [2]:
def synthetic_data(w,b,num_examples):
    """生成y = Xw + b + 噪声"""
    X=torch.normal(0,1,(num_examples,len(w)))
    y=torch.matmul(X,w)+b
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

true_w=torch.tensor([2,-3.4])
true_b=4.2
features,labels=synthetic_data(true_w,true_b,1000)

调用框架中现有的API来读取数据

In [4]:
def load_array(data_arrays,batch_size,is_train=True):
    """构造一个pytorch数据迭代器"""
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)
#DataLoader中参数表示在dataset中选取打乱顺序的batch_size个数据
batch_size=10
data_iter=load_array((features,labels),batch_size)
next(iter(data_iter))

[tensor([[ 1.6268,  1.8837],
         [-0.1251,  0.7440],
         [ 0.6159, -0.2940],
         [ 0.2327, -0.0977],
         [ 0.3851, -1.5330],
         [ 0.7350, -0.3827],
         [ 0.7884,  0.1748],
         [-0.0472,  0.4832],
         [ 0.1828, -0.9914],
         [ 1.6920,  0.6055]]),
 tensor([[ 1.0548],
         [ 1.4137],
         [ 6.4275],
         [ 5.0032],
         [10.1578],
         [ 6.9711],
         [ 5.1617],
         [ 2.4638],
         [ 7.9194],
         [ 5.5246]])]

使用框架的预定义好的层

In [6]:
#nn是神经网络的缩写
from torch import nn

net=nn.Sequential(nn.Linear(2,1))#nn.Linear表示网络的线性层，表示输入为2，输出为1
#Sequential相当于一个容器，将层放好

初始化模型参数

In [7]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是MSELoss类，也称为L2范数

In [8]:
loss=nn.MSELoss()

实例化SGD实例

In [9]:
trainer=torch.optim.SGD(net.parameters(),lr=0.03)

训练过程代码与我们之前所做的相似

In [10]:
num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l=loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l=loss(net(features),labels)
    print(f'epoch{epoch+1},loss{l:f}')

epoch1,loss0.000522
epoch2,loss0.000103
epoch3,loss0.000104
